In [311]:
import sys
import spotipy
import spotipy.util as util

scopes = 'user-library-read user-read-recently-played playlist-modify-public'

token = util.prompt_for_user_token(username='',scope=scopes,client_id='635ce8d7c7894bc192bab6a86d85eaff',client_secret='17f44a98289b42b6be9a464ee4bc35e1',redirect_uri='http://www.google.com/')
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print "Can't get token for", username

In [17]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score as roc
from sklearn.metrics import roc_curve
from sklearn.neighbors import NearestNeighbors

In [229]:
datasetPlaylist = sp.user_playlist(user="rishik34561",playlist_id="4RS8meV33aCDaJHQmKV4OD")

u'4RS8meV33aCDaJHQmKV4OD'

In [19]:
sourcePlaylist = sp.user_playlist_tracks(user="rishik34561",playlist_id="2ZLQ8UzWUFxlRsymP6tuU7")
sourcePlaylistTracks = sourcePlaylist['items']
while sourcePlaylist['next']:
    sourcePlaylist = sp.next(sourcePlaylist)
    sourcePlaylistTracks.extend(sourcePlaylist['items'])

[{u'added_at': u'2017-03-09T07:29:00Z',
  u'added_by': {u'external_urls': {u'spotify': u'https://open.spotify.com/user/rishik34561'},
   u'href': u'https://api.spotify.com/v1/users/rishik34561',
   u'id': u'rishik34561',
   u'type': u'user',
   u'uri': u'spotify:user:rishik34561'},
  u'is_local': False,
  u'primary_color': None,
  u'track': {u'album': {u'album_type': u'album',
    u'artists': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/1URnnhqYAYcrqrcwql10ft'},
      u'href': u'https://api.spotify.com/v1/artists/1URnnhqYAYcrqrcwql10ft',
      u'id': u'1URnnhqYAYcrqrcwql10ft',
      u'name': u'21 Savage',
      u'type': u'artist',
      u'uri': u'spotify:artist:1URnnhqYAYcrqrcwql10ft'}],
    u'available_markets': [u'AD',
     u'AE',
     u'AR',
     u'AT',
     u'AU',
     u'BE',
     u'BG',
     u'BH',
     u'BO',
     u'BR',
     u'CA',
     u'CH',
     u'CL',
     u'CO',
     u'CR',
     u'CY',
     u'CZ',
     u'DE',
     u'DK',
     u'DO',
     u'DZ',
     u'

In [14]:
trackIds = []
i = 0
print(len(sourcePlaylistTracks))
for item in sourcePlaylistTracks:
    trackIds.append(item['track']['uri'])
    i = i + 1

i = 0
for i in range(len(trackIds)):
    sp.user_playlist_add_tracks('rishik34561',datasetPlaylist['id'],[trackIds[i]])


376


In [24]:
sourcePlaylist = sp.user_playlist_tracks(user="spotify",playlist_id="37i9dQZF1DX1YPTAhwehsC")
sourcePlaylistTracks = sourcePlaylist['items']
while sourcePlaylist['next']:
    sourcePlaylist = sp.next(sourcePlaylist)
    sourcePlaylistTracks.extend(sourcePlaylist['items'])

trackIds = []
i = 0
print(len(sourcePlaylistTracks))
for item in sourcePlaylistTracks:
    trackIds.append(item['track']['uri'])
    i = i + 1

i = 0
for i in range(len(trackIds)):
    sp.user_playlist_add_tracks('rishik34561',datasetPlaylist['id'],[trackIds[i]])



50


In [25]:
datasetPlaylistTracks = datasetPlaylist['items']
while datasetPlaylist['next']:
    datasetPlaylist = sp.next(datasetPlaylist)
    datasetPlaylistTracks.extend(datasetPlaylist['items'])
datasetPlaylistTracks

trackIds = []
i = 0
print(len(datasetPlaylistTracks))
for item in datasetPlaylistTracks:
    trackIds.append(item['track']['uri'])
    i = i + 1




1816


In [26]:
datasetFeatures = []
j = 0
for i in range(0,len(trackIds),50):
    audio_features = sp.audio_features(trackIds[i:i+50])
    for track in audio_features:
        track['song_title'] = datasetPlaylistTracks[j]['track']['name']
        track['artist'] = datasetPlaylistTracks[j]['track']['artists'][0]['name']
        j = j + 1
        datasetFeatures.append(track)


1816

In [242]:
datasetDf = pd.DataFrame(datasetFeatures)

In [305]:
datasetDf.drop_duplicates(subset='id', keep='first', inplace=True)
datasetDf.head()

,acousticness,analysis_url,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,song_title,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.128000,https://api.spotify.com/v1/audio-analysis/3xAX...,21 Savage,0.726,151615,0.597,3xAXYCtd98G5G0SSJAHtqd,0.03260,7,0.1310,-8.574,1,Dip Dip,0.145,83.021,4,https://api.spotify.com/v1/tracks/3xAXYCtd98G5...,audio_features,spotify:track:3xAXYCtd98G5G0SSJAHtqd,0.367
1,0.220000,https://api.spotify.com/v1/audio-analysis/4go2...,A$AP Rocky,0.680,192067,0.843,4go2hxLM6ijk0K76ZY0Nhd,0.00003,1,0.3780,-4.919,1,Goldie,0.241,81.958,4,https://api.spotify.com/v1/tracks/4go2hxLM6ijk...,audio_features,spotify:track:4go2hxLM6ijk0K76ZY0Nhd,0.743
2,0.167000,https://api.spotify.com/v1/audio-analysis/5sl2...,A$AP Rocky,0.527,170307,0.831,5sl2735QKaOmL8UL4Q39Ut,0.00000,10,0.0861,-6.876,0,Peso,0.270,140.017,4,https://api.spotify.com/v1/tracks/5sl2735QKaOm...,audio_features,spotify:track:5sl2735QKaOmL8UL4Q39Ut,0.584
3,0.000052,https://api.spotify.com/v1/audio-analysis/5bUw...,Drake,0.487,212613,0.712,5bUwUQKGNnTE20kb3MTqHD,0.00000,8,0.1980,-7.323,1,Under Ground Kings,0.270,156.209,4,https://api.spotify.com/v1/tracks/5bUwUQKGNnTE...,audio_features,spotify:track:5bUwUQKGNnTE20kb3MTqHD,0.397
4,0.270000,https://api.spotify.com/v1/audio-analysis/4QRW...,Logic,0.709,287400,0.787,4QRWA5UqYU1st9aO0UfxOj,0.00114,1,0.2960,-2.377,1,Fade Away,0.235,75.986,4,https://api.spotify.com/v1/tracks/4QRWA5UqYU1s...,audio_features,spotify:track:4QRWA5UqYU1st9aO0UfxOj,0.607


In [290]:
recentTracks = sp.current_user_recently_played(limit=5)
recentSongs = recentTracks['items']


In [292]:
recentIds = []
for song in recentSongs:
    recentIds.append(song['track']['id'])
print(len(recentIds))

recentFeatures = []
k = 0
for i in range(len(recentIds)):
    audio_features = sp.audio_features([recentIds[i]])
    if audio_features == [None]:
        continue
    for j in range(len(audio_features)):
        audio_features[j]['song_title'] = recentSongs[k]['track']['name']
        audio_features[j]['artist'] = recentSongs[k]['track']['artists'][0]['name']
        k = k + 1
        recentFeatures.append(audio_features[j])


3


In [293]:
datasetFeatures.extend(recentFeatures)

In [294]:
datasetDf = pd.DataFrame(datasetFeatures)
datasetDf.drop_duplicates(subset='id', keep='first', inplace=True)
datasetDf.reset_index(inplace=True,drop=True)

In [295]:
datasetDf.tail()

,acousticness,analysis_url,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,song_title,speechiness,tempo,time_signature,track_href,type,uri,valence
1763,0.33200,https://api.spotify.com/v1/audio-analysis/4mOP...,88GLAM,0.714,204615,0.630,4mOPVJGfWXH0wPFA6xeRDz,0.000002,9,0.110,-5.690,0,Kyrie,0.0653,156.112,4,https://api.spotify.com/v1/tracks/4mOPVJGfWXH0...,audio_features,spotify:track:4mOPVJGfWXH0wPFA6xeRDz,0.3660
1764,0.37200,https://api.spotify.com/v1/audio-analysis/3ZOw...,88GLAM,0.775,211862,0.539,3ZOwmSmQKX4vTK4N6e2GsW,0.000008,5,0.225,-4.303,1,Cake,0.0881,144.966,4,https://api.spotify.com/v1/tracks/3ZOwmSmQKX4v...,audio_features,spotify:track:3ZOwmSmQKX4vTK4N6e2GsW,0.6240
1765,0.07080,https://api.spotify.com/v1/audio-analysis/2YhB...,Big Sean,0.725,303280,0.606,2YhBJbeXBz2VsthtV5O4bY,0.000000,2,0.215,-6.296,1,Nothing Is Stopping You,0.1680,140.015,4,https://api.spotify.com/v1/tracks/2YhBJbeXBz2V...,audio_features,spotify:track:2YhBJbeXBz2VsthtV5O4bY,0.2060
1766,0.00979,https://api.spotify.com/v1/audio-analysis/6w2V...,Tory Lanez,0.480,339520,0.523,6w2VaBGWcColW05nZROWvS,0.000000,7,0.143,-8.304,1,To D.R.E.A.M.,0.1050,137.671,5,https://api.spotify.com/v1/tracks/6w2VaBGWcCol...,audio_features,spotify:track:6w2VaBGWcColW05nZROWvS,0.0979
1767,0.05460,https://api.spotify.com/v1/audio-analysis/5jlL...,Swae Lee,0.784,225037,0.416,5jlLBqK31dOqXECKn3jlMX,0.000022,7,0.112,-9.231,1,Sextasy,0.1020,147.994,4,https://api.spotify.com/v1/tracks/5jlLBqK31dOq...,audio_features,spotify:track:5jlLBqK31dOqXECKn3jlMX,0.2590


In [296]:
datasetDfWithoutText = pd.DataFrame(datasetFeatures)
datasetDfWithoutText.drop_duplicates(subset='id', keep='first', inplace=True)
datasetDfWithoutText.drop(['artist','song_title','track_href','type','analysis_url','uri','id'],axis=1, inplace=True)

In [297]:
datasetDfWithoutText.tail()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
1829,0.33200,0.714,204615,0.630,0.000002,9,0.110,-5.690,0,0.0653,156.112,4,0.3660
1831,0.37200,0.775,211862,0.539,0.000008,5,0.225,-4.303,1,0.0881,144.966,4,0.6240
1833,0.07080,0.725,303280,0.606,0.000000,2,0.215,-6.296,1,0.1680,140.015,4,0.2060
1835,0.00979,0.480,339520,0.523,0.000000,7,0.143,-8.304,1,0.1050,137.671,5,0.0979
1836,0.05460,0.784,225037,0.416,0.000022,7,0.112,-9.231,1,0.1020,147.994,4,0.2590


In [298]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='auto').fit(datasetDfWithoutText)

In [299]:
distances, indices = nbrs.kneighbors(datasetDfWithoutText)
print(indices)

[[   0  243 1290  255  832  584]
 [   1  987  286 1563  157  115]
 [   2 1598  234 1714 1360 1590]
 ...
 [1765   22 1128 1509 1650  903]
 [1766  485 1677 1505 1068  430]
 [1767 1185 1456  158 1420 1169]]


In [300]:
smartPlaylist = sp.user_playlist_create('rishik34561', "Smart Song Suggestions", True, "Similar songs to the ones you have been listening to recently")

In [301]:
for recentFeature in recentFeatures:
    rowNum = datasetDf[datasetDf["id"]==recentFeature['id']].index.tolist()[0]
    print("Similar songs to " + datasetDf.iloc[rowNum]['song_title'] + ":")
    for songId in indices[rowNum][1:]:
        # print similar tracks to the recently listened songs
        print(datasetDf.iloc[songId]['song_title'] + " by " + datasetDf.iloc[songId]['artist'])
        sp.user_playlist_add_tracks('rishik34561',smartPlaylist['id'],[datasetDf.iloc[songId]['id']])



(u'Impossible', ' similar songs:')
T-Shirt by Migos
Skateboard P (feat. Big Sean) by MadeinTYO
Just Give Me a Reason (feat. Nate Ruess) by P!nk
Calling All Angels by Train
I'm Yours by Jason Mraz
(u'To D.R.E.A.M.', ' similar songs:')
90210 (feat. Kacy Hill) by Travis Scott
Star Roving by Slowdive
Joe Frazier by Orions Belte
Sign of the Times by Harry Styles
Buried Alive by Logic
(u'Sextasy', ' similar songs:')
Like I'm Gonna Lose You (feat. John Legend) by Meghan Trainor
4-U by Lucys
Ridin (feat. Landon Cube & YBN Nahmir) by Yung Bans
Paint Me Silver by Pond
New Soul by Yael Naim
